# Introducere în Probabilitate și Statistică
În acest caiet, ne vom juca cu unele dintre conceptele pe care le-am discutat anterior. Multe concepte din probabilitate și statistică sunt bine reprezentate în biblioteci majore pentru procesarea datelor în Python, precum `numpy` și `pandas`.


In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

## Variabile aleatorii și distribuții
Să începem prin a extrage un eșantion de 30 de valori dintr-o distribuție uniformă de la 0 la 9. Vom calcula, de asemenea, media și varianța.


In [ ]:
sample = [ random.randint(0,10) for _ in range(30) ]
print(f"Sample: {sample}")
print(f"Mean = {np.mean(sample)}")
print(f"Variance = {np.var(sample)}")

Pentru a estima vizual câte valori diferite există în eșantion, putem realiza un **histogramă**:


In [ ]:
plt.hist(sample)
plt.show()

## Analiza datelor reale

Media și varianța sunt foarte importante atunci când se analizează date din lumea reală. Să încărcăm datele despre jucătorii de baseball de la [SOCR MLB Height/Weight Data](http://wiki.stat.ucla.edu/socr/index.php/SOCR_Data_MLB_HeightsWeights)


In [ ]:
df = pd.read_csv("../../data/SOCR_MLB.tsv",sep='\t', header=None, names=['Name','Team','Role','Weight','Height','Age'])
df


> Folosim un pachet numit [**Pandas**](https://pandas.pydata.org/) aici pentru analiza datelor. Vom vorbi mai multe despre Pandas și lucrul cu date în Python mai târziu în acest curs.

Să calculăm valorile medii pentru vârstă, înălțime și greutate:


In [ ]:
df[['Age','Height','Weight']].mean()

Acum să ne concentrăm pe înălțime și să calculăm deviația standard și varianța:


In [ ]:
print(list(df['Height'])[:20])

In [ ]:
mean = df['Height'].mean()
var = df['Height'].var()
std = df['Height'].std()
print(f"Mean = {mean}\nVariance = {var}\nStandard Deviation = {std}")

Pe lângă medie, este util să analizăm valoarea mediană și categoriile quartile. Acestea pot fi vizualizate utilizând un **diagramă cutie**:


In [ ]:
plt.figure(figsize=(10,2))
plt.boxplot(df['Height'].ffill(), vert=False, showmeans=True)
plt.grid(color='gray', linestyle='dotted')
plt.tight_layout()
plt.show()

Putem, de asemenea, să realizăm diagrame box-plot pentru subseturi ale setului nostru de date, de exemplu, grupate după rolul jucătorului.


In [ ]:
df.boxplot(column='Height', by='Role', figsize=(10,8))
plt.xticks(rotation='vertical')
plt.tight_layout()
plt.show()

> **Notă**: Acest diagram sugerează că, în medie, înălțimile primilor basiști sunt mai mari decât înălțimile celor de la al doilea bază. Mai târziu vom învăța cum putem testa această ipoteză mai formal și cum să demonstrăm că datele noastre sunt statistic semnificative pentru a arăta acest lucru.

Vârsta, înălțimea și greutatea sunt toate variabile aleatoare continue. Ce crezi că este distribuția lor? O metodă bună de a afla este să reprezentăm histograma valorilor:


In [ ]:
df['Weight'].hist(bins=15, figsize=(10,6))
plt.suptitle('Weight distribution of MLB Players')
plt.xlabel('Weight')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

## Distribuția Normală

Să creăm un eșantion artificial de greutăți care urmează o distribuție normală cu aceeași medie și variață ca și datele noastre reale:


In [ ]:
generated = np.random.normal(mean, std, 1000)
generated[:20]

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(generated, bins=15)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(np.random.normal(0,1,50000), bins=300)
plt.tight_layout()
plt.show()

Deoarece majoritatea valorilor în viața reală sunt distribuite normal, nu ar trebui să folosim un generator de numere aleatoare uniforme pentru a genera date de probă. Iată ce se întâmplă dacă încercăm să generăm greutăți cu o distribuție uniformă (generată de `np.random.rand`):


In [ ]:
wrong_sample = np.random.rand(1000)*2*std+mean-std
plt.figure(figsize=(10,6))
plt.hist(wrong_sample)
plt.tight_layout()
plt.show()

## Intervale de încredere

Să calculăm acum intervalele de încredere pentru greutățile și înălțimile jucătorilor de baseball. Vom folosi codul [din această discuție pe stackoverflow](https://stackoverflow.com/questions/15033511/compute-a-confidence-interval-from-sample-data):


In [ ]:
import scipy.stats

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h

for p in [0.85, 0.9, 0.95]:
    m, h = mean_confidence_interval(df['Weight'].fillna(method='pad'),p)
    print(f"p={p:.2f}, mean = {m:.2f} ± {h:.2f}")

## Testarea ipotezelor

Să explorăm diferite roluri în setul nostru de date al jucătorilor de baseball:


In [ ]:
df.groupby('Role').agg({ 'Weight' : 'mean', 'Height' : 'mean', 'Age' : 'count'}).rename(columns={ 'Age' : 'Count'})

Să testăm ipoteza că Prima bază este mai înaltă decât a Doua bază. Cel mai simplu mod de a face acest lucru este să testăm intervalele de încredere:


In [ ]:
for p in [0.85,0.9,0.95]:
    m1, h1 = mean_confidence_interval(df.loc[df['Role']=='First_Baseman',['Height']],p)
    m2, h2 = mean_confidence_interval(df.loc[df['Role']=='Second_Baseman',['Height']],p)
    print(f'Conf={p:.2f}, 1st basemen height: {m1-h1[0]:.2f}..{m1+h1[0]:.2f}, 2nd basemen height: {m2-h2[0]:.2f}..{m2+h2[0]:.2f}')

Putem vedea că intervalele nu se suprapun.

O metodă statistic mai corectă de a demonstra ipoteza este să folosim un **test t Student**:


In [ ]:
from scipy.stats import ttest_ind

tval, pval = ttest_ind(df.loc[df['Role']=='First_Baseman',['Height']], df.loc[df['Role']=='Second_Baseman',['Height']],equal_var=False)
print(f"T-value = {tval[0]:.2f}\nP-value: {pval[0]}")

Cele două valori returnate de funcția `ttest_ind` sunt:
* valoarea p poate fi considerată ca probabilitatea ca două distribuții să aibă aceeași medie. În cazul nostru, este foarte mică, ceea ce înseamnă că există dovezi puternice care susțin că jucătorii de primă bază sunt mai înalți.
* valoarea t este valoarea intermediară a diferenței normale a mediilor utilizată în testul t și este comparată cu o valoare prag pentru un anumit nivel de încredere.


## Simularea unei distribuții normale cu Teorema Limitei Centrale

Generatorul pseudo-aleator din Python este conceput să ne ofere o distribuție uniformă. Dacă vrem să creăm un generator pentru distribuția normală, putem folosi teorema limitei centrale. Pentru a obține o valoare distribuită normal, vom calcula pur și simplu media unui eșantion generat uniform.


In [ ]:
def normal_random(sample_size=100):
    sample = [random.uniform(0,1) for _ in range(sample_size) ]
    return sum(sample)/sample_size

sample = [normal_random() for _ in range(100)]
plt.figure(figsize=(10,6))
plt.hist(sample)
plt.tight_layout()
plt.show()

## Corelație și Evil Baseball Corp

Corelația ne permite să găsim relații între secvențe de date. În exemplul nostru simplu, să presupunem că există o corporație malefică de baseball care plătește jucătorii în funcție de înălțimea lor - cu cât jucătorul este mai înalt, cu atât primește mai mulți bani. Să presupunem că există un salariu de bază de 1000 USD, și o primă suplimentară de la 0 la 100 USD, în funcție de înălțime. Vom lua jucătorii reali din MLB și vom calcula salariile lor imaginare:


In [ ]:
heights = df['Height'].fillna(method='pad')
salaries = 1000+(heights-heights.min())/(heights.max()-heights.mean())*100
print(list(zip(heights, salaries))[:10])

Să calculăm acum covarianța și corelația acelor secvențe. `np.cov` ne va oferi o așa-numită **matrice de covarianță**, care este o extensie a covarianței pentru variabile multiple. Elementul $M_{ij}$ al matricei de covarianță $M$ este o corelație între variabilele de intrare $X_i$ și $X_j$, iar valorile de pe diagonală $M_{ii}$ reprezintă varianța lui $X_{i}$. În mod similar, `np.corrcoef` ne va oferi **matricea de corelație**.


In [ ]:
print(f"Covariance matrix:\n{np.cov(heights, salaries)}")
print(f"Covariance = {np.cov(heights, salaries)[0,1]}")
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

O corelație egală cu 1 înseamnă că există o **relație liniară** puternică între două variabile. Putem vedea vizual relația liniară prin trasarea unui grafic cu o valoare în funcție de cealaltă:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights,salaries)
plt.tight_layout()
plt.show()

Să vedem ce se întâmplă dacă relația nu este liniară. Să presupunem că corporația noastră a decis să ascundă dependența liniară evidentă dintre înălțimi și salarii și a introdus o anumită non-liniaritate în formulă, cum ar fi `sin`:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

În acest caz, corelația este puțin mai mică, dar este totuși destul de ridicată. Acum, pentru a face relația și mai puțin evidentă, am putea dori să adăugăm o oarecare aleatorietate suplimentară prin adăugarea unei variabile aleatoare la salariu. Să vedem ce se întâmplă:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100+np.random.random(size=len(heights))*20-10
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights, salaries)
plt.tight_layout()
plt.show()

> Poți ghici de ce punctele se aliniază în linii verticale astfel?

Am observat corelația dintre un concept supus unui proces artificial, precum salariul, și variabila observată *înălțimea*. Să vedem dacă cele două variabile observate, cum ar fi înălțimea și greutatea, sunt corelate:


In [ ]:
np.corrcoef(df['Height'].ffill(),df['Weight'])

Din păcate, nu am obținut niciun rezultat - doar câteva valori ciudate `nan`. Acest lucru se datorează faptului că unele dintre valorile din seria noastră sunt nedefinite, reprezentate ca `nan`, ceea ce face ca rezultatul operației să fie de asemenea nedefinit. Privind matricea putem vedea că `Weight` este coloana problematică, deoarece a fost calculată autocorelarea între valorile `Height`.

> Acest exemplu evidențiază importanța **pregătirii datelor** și **curățării**. Fără date corespunzătoare nu putem calcula nimic.

Să folosim metoda `fillna` pentru a completa valorile lipsă și să calculăm corelația: 


In [ ]:
np.corrcoef(df['Height'].fillna(method='pad'), df['Weight'])

Există într-adevăr o corelație, dar nu atât de puternică ca în exemplul nostru artificial. Într-adevăr, dacă ne uităm la diagrama de dispersie a unei valori față de cealaltă, relația ar fi mult mai puțin evidentă:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(df['Weight'],df['Height'])
plt.xlabel('Weight')
plt.ylabel('Height')
plt.tight_layout()
plt.show()

## Concluzie

În acest caiet am învățat cum să efectuăm operații de bază asupra datelor pentru a calcula funcții statistice. Acum știm cum să folosim un aparat solid de matematică și statistică pentru a demonstra unele ipoteze și cum să calculăm intervale de încredere pentru variabile arbitrare date un eșantion de date.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Declinare a responsabilității**:  
Acest document a fost tradus folosind serviciul de traducere AI [Co-op Translator](https://github.com/Azure/co-op-translator). Deși ne străduim pentru acuratețe, vă rugăm să rețineți că traducerile automate pot conține erori sau inexactități. Documentul original, în limba sa nativă, trebuie considerat sursa oficială. Pentru informații critice, se recomandă traducerea profesională realizată de un specialist uman. Nu ne asumăm răspunderea pentru eventuale neînțelegeri sau interpretări greșite rezultate din folosirea acestei traduceri.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
